In [3]:
#IMPORTS

#File IO
import os
import glob

#Data manipulation
import numpy as np

#Pytorch
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

#Misc
from tqdm import tqdm


In [4]:
#GET TARGET DATA

def get_features(file):
    with np.load(file) as data:
        bands = data['rad']
        features = np.stack([
            data['l2_cloud_mask'],
            data['l2_cloud_phase'],
            np.log1p(data['l2_cod']),  # log(1 + cod)
            np.log1p(data['l2_cps'])   # log(1 + cps)
        ])
    return bands, features

In [5]:
shared_files = ['/umbc/rs/nasa-access/users/xingyan/pytorch-caney/data/downstream_2dcloud_scaled/ABI_Data_20231451250225_1577.npz', '/umbc/rs/nasa-access/users/xingyan/pytorch-caney/data/downstream_2dcloud_scaled/ABI_Data_20231482000207_932.npz', '/umbc/rs/nasa-access/users/xingyan/pytorch-caney/data/downstream_2dcloud_scaled/ABI_Data_20231111440210_186.npz', '/umbc/rs/nasa-access/users/xingyan/pytorch-caney/data/downstream_2dcloud_scaled/ABI_Data_20231701140225_465.npz', '/umbc/rs/nasa-access/users/xingyan/pytorch-caney/data/downstream_2dcloud_scaled/ABI_Data_20231541010218_855.npz', '/umbc/rs/nasa-access/users/xingyan/pytorch-caney/data/downstream_2dcloud_scaled/ABI_Data_20231470940228_463.npz', '/umbc/rs/nasa-access/users/xingyan/pytorch-caney/data/downstream_2dcloud_scaled/ABI_Data_20231491330208_399.npz', '/umbc/rs/nasa-access/users/xingyan/pytorch-caney/data/downstream_2dcloud_scaled/ABI_Data_20231051950204_363.npz', '/umbc/rs/nasa-access/users/xingyan/pytorch-caney/data/downstream_2dcloud_scaled/ABI_Data_20231241550212_1569.npz']

shared_files = [f[80:] for f in shared_files]

shared_files[0]

'ABI_Data_20231451250225_1577.npz'

In [6]:
file_list = glob.glob('../ABI_Data_Scaled/*.npz')
file_list[0][19:]

'ABI_Data_20231541040218_855.npz'

In [7]:
max_images = 15000

#CREATE DATASET

file_list = glob.glob('../ABI_Data_Scaled/*.npz')

X = []
y = []

for i, file in tqdm(enumerate(file_list)):
    features, target = get_features(file)
    X.append(features)
    y.append(target)
    if (file[19:] in shared_files):
        print(f"Shared file: {file}")
        continue
    if i == max_images-1: break

X = np.stack(X)  # shape: (Samples, 128, 128, 16)
X = torch.from_numpy(X).float()
X = X.permute(0, 3, 1, 2)  # shape: (Samples, 16, 128, 128)


y = np.stack(y)
y = torch.from_numpy(y).float() # shape: (Samples, 4, 128, 128)

dataset = TensorDataset(X, y)
torch.save(dataset, "dataset_scaled.pt")

563it [00:38, 14.73it/s] 


KeyboardInterrupt: 